In [1]:
# transformers 패키지 설치
# !pip install transformers==4.50.0 datasets==3.5.0 huggingface_hub==0.29.0 -qqq 
# !pip install --upgrade transformers tokenizers

In [2]:
# BERT 와 GPT-2 모델을 활용할때 허깅페이스 트랜스포머 코드 비교

# 허깅페이스 트렌스포머를 활용하면 서로 다른 조직에서 개발한 BERT와 GPT-2 모델을 거의 동인한 인터페이스로 활용 가능
# AutoModel, AutoTokenizer 클래스를 사용해 BERT 및 GPT-2 모델과 토큰나이저를 불러오고 토큰화를 수행해서 모델에 입력으로 넣어준다
from transformers import AutoModel, AutoTokenizer

text = 'What is Huggingface Transformers?'

# BERT 모델 활용
bert_model = AutoModel.from_pretrained('bert-base-uncased') # pre-trained model(BERT) 불러오기
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') # 토크나이저 불러오기
encoded_input =  bert_tokenizer(text, return_tensors='pt') # 입력 토큰화
bert_output = bert_model(**encoded_input) # 모델에 입력

2025-09-02 15:51:11.056924: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# GPT-2 모델 활용
gpt_model = AutoModel.from_pretrained('gpt2') # pre-trained model(GPT) 불러오기
gpt_tokenizer = AutoTokenizer.from_pretrained('gpt2') # 토크나이저 불러오기
encoded_input = gpt_tokenizer(text, return_tensors='pt') # 입력 토큰화
gpt_output = gpt_model(**encoded_input) # 모델에 입력

In [4]:
# 허깅페이스 트랜스포머 라미이브러리
# - 허깅페이스에는 모델을 바디(body), 헤드(head)로 구분, 같은 바디를 사용하면서 다른 작업에 사용 할 수 이도록 만들기 위함
# 예) 구글 BERT 모델을 사용하지만 다른 헤드를 사용 할 수가 있다
# - 문장 전체가 긍정인지 부정인지를 분류하는 모델, 이때는 바디가 반환하는 여러 잠재 상태(hidden state)중 가장 앞에 있는 [CLS] 토큰의 데이터만 받아 예측
# - 각 토큰이 사람이나 장소에 해당하는지 판단하는 개처명 인식(named entity recognition) 모델인데, 각 토큰에 대해 판단해야 하기 때문에 모든 토튼의
# - 데이터를 받아 각각 사람인지 장소인지를 예측

# 모델 아이디로 모델 불러오기 코드 - 모댈의 바디만 불어오는 코드
from transformers import AutoModel

model_id = 'klue/roberta-base'
model = AutoModel.from_pretrained(model_id)
model.save_pretrained('./transformers_models/klue/roberta-base')


Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# 분류 헤드가 포함된 모델 불러오기 - SamLowe/roberta-base-go_emotions 저장소에서 텍스트 분류 모델을 내려 받아 classification_model 변수에 저장
# SamLowe/roberta-base-go_emotions 모델은 분류 헤드가 포함(입력 문장이 어떤 감성을 나타내는지 분류하는데, 감탄/즐거움/화 등과 같은 감정이 포함되어 있는지 분류)

from transformers import AutoModelForSequenceClassification
model_id = 'SamLowe/roberta-base-go_emotions'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)
classification_model.save_pretrained('./transformers_models/SamLowe/roberta-base-go_emotions')

In [6]:
# 분류 헤드가 랜덤으로 초기화된 모델 불러오기
from transformers import AutoModelForSequenceClassification

model_id = 'klue/roberta-base'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)
classification_model.save_pretrained('./transformers_models/klue/roberta-base')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 토큰나이저 불러오기
# AutoTokenizer 클래스를 통해 앞서 모델을 불러올때도 사용한 klue/roberta-base 저장소의 토큰나이저를 불러온다
# config_json 파일에서 확인

from transformers import AutoTokenizer

model_id = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
# 토크나이저 사용하기

# input_ids는 토큰화했을때 각 토큰이 토크나이저 사전에 몇번째 항목인지 나타냄
# input_ids의 첫번째 항목은 0이고 두번째 항목은 9157인데 각각 [CLS]와 토크에 대응되는 것을 확인 할 수 있다
tokenized = tokenizer('토큰나이저는 텍스트를 토큰 단위로 나눈다.')
print(tokenized)
# {'input_ids': [0, 1793, 2855, 7461, 2190, 2259, 8509, 2138, 1793, 2855, 5385, 2200, 20950, 18, 2], 
# 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
# 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

# attention mask가 1이면 패딩이 아닌 실제 토큰임을 의미, token_type_ids가 0이면 일반적으로 첫번째 문장임을 의미
# attention mask는 실제 텍스트 길이를 맞추기 위해 추가한 패딩인지 알려주는 mask
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids']))
# ['[CLS]', '토', '##큰', '##나이', '##저', '##는', '텍스트', '##를', '토', '##큰', '단위', '##로', '나눈다', '.', '[SEP]']

# 토큰 아이디를 다시 텍스트로 돌리고 싶다면 토크나이저의 decode 메서드를 사용
print(tokenizer.decode(tokenized['input_ids']))
# [CLS] 토큰나이저는 텍스트를 토큰 단위로 나눈다. [SEP]

# 특수 문자 토큰을 제외하고 싶다면 skip_special_tokens=True 설정
print(tokenizer.decode(tokenized['input_ids'], skip_special_tokens=True))

{'input_ids': [0, 1793, 2855, 7461, 2190, 2259, 8509, 2138, 1793, 2855, 5385, 2200, 20950, 18, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', '토', '##큰', '##나이', '##저', '##는', '텍스트', '##를', '토', '##큰', '단위', '##로', '나눈다', '.', '[SEP]']
[CLS] 토큰나이저는 텍스트를 토큰 단위로 나눈다. [SEP]
토큰나이저는 텍스트를 토큰 단위로 나눈다.


In [9]:
# 토크나이저에 여러 문장 넣기
# 토크나이저에 한번에 여러 문장을 처리 할 수도 있다, '첫 번째 문장', '두 번째 문장'을 리스트로 함께 넣었다
# 출력 결과를 확인하면 모두 각 문장을 토큰화해 2개 리스트를 반환
tokenizer(['첫 번째 문장', '두 번째 문장'])
# {'input_ids': [[0, 1656, 1141, 3135, 6265, 2], [0, 864, 1141, 3135, 6265, 2]], 
# 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 
# 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

{'input_ids': [[0, 1656, 1141, 3135, 6265, 2], [0, 864, 1141, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [10]:
# 하나의 데이터에 여러 문장이 들어가는 경우
# 한번에 2개의 문장을 모델에 넣어야 하는 경우 - 2개의 문장이 서로 원인과 결과 관계인지 학습시키기 싶다면, 두 문장을 한번에 모델에 입력해야됨
# 이때 2개의 문장이 하나의 데이터라는 것을 표시하기 위해 리스트로 한번더 감싸준다
tokenizer([ ['첫 번째 문장', '두 번째 문장'] ])
# {'input_ids': [[0, 1656, 1141, 3135, 6265, 2, 864, 1141, 3135, 6265, 2]], 
# 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 
# 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

{'input_ids': [[0, 1656, 1141, 3135, 6265, 2, 864, 1141, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [11]:
# 토큰 아이디를 문자열로 복원

# 토크나이저의 batch_decode() 메서드를 사용하면 input_ids부분의 토큰 아이디를 문자열로 복월 할 수 있는데
# 기본적으로 토큰화를 하면 ['[CLS] 첫 번째 문장 [SEP]', '[CLS] 두 번째 문장 [SEP]']
# 2개의 문자을 한번에 토큰화하면 ['[CLS] 첫 번째 문장 [SEP] 두 번째 문장 [SEP]']

first_tokenized_result = tokenizer( ['첫 번째 문장', '두 번째 문장'] )['input_ids']
print(first_tokenized_result)
print(tokenizer.batch_decode(first_tokenized_result))

second_tokenized_result = tokenizer( [ ['첫 번째 문장', '두 번째 문장'] ] )['input_ids']
print(second_tokenized_result)
print(tokenizer.batch_decode(second_tokenized_result))

[[0, 1656, 1141, 3135, 6265, 2], [0, 864, 1141, 3135, 6265, 2]]
['[CLS] 첫 번째 문장 [SEP]', '[CLS] 두 번째 문장 [SEP]']
[[0, 1656, 1141, 3135, 6265, 2, 864, 1141, 3135, 6265, 2]]
['[CLS] 첫 번째 문장 [SEP] 두 번째 문장 [SEP]']


In [12]:
# BERT 토크나이저와 RoBERTa 토크나이저

# BERT는 학습할때 2개의 문장이 서로 이어지는지 맞추는 NSP(next sentence prediction) 작업을 수행하기위해 문장을 구분하는 토큰 타입 아이디를 만들었다
# 첫번째 문장의 토큰 아이디는 0, 두번째 문장의 토큰 아이디 1
bert_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
print(bert_tokenizer( [ ['첫 번째 문장', '두 번째 문장'] ] ))
# {'input_ids': [[2, 1656, 1141, 3135, 6265, 3, 864, 1141, 3135, 6265, 3]], 
# 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]], 
# 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

# RoBERTa 모델의 경우 NSP 작업을 학습 과정에서 제거
roberta_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
print(roberta_tokenizer( [ ['첫 번째 문장', '두 번째 문장'] ] ))
# {'input_ids': [[0, 1656, 1141, 3135, 6265, 2, 864, 1141, 3135, 6265, 2]], 
# 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 
# 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

# 실제 영어 버전의 roberta-base 모델에는 항목 자체가 없음
en_roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-base')
print(en_roberta_tokenizer( [ ['first sentence', 'second sentence'] ] ))
# {'input_ids': [[0, 9502, 3645, 2, 2, 10815, 3645, 2]], 
# 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}

{'input_ids': [[2, 1656, 1141, 3135, 6265, 3, 864, 1141, 3135, 6265, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [[0, 1656, 1141, 3135, 6265, 2, 864, 1141, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [[0, 9502, 3645, 2, 2, 10815, 3645, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}


In [13]:
# attention_mask 확인
# 해당 토큰이 패딩 토큰인지 실제 데이터인지에 대한 정보, 
# 패딩은 모델에 입력하는 토큰 아이디의 길이를 맞추기 위해 추가하는 토큰, padding='longest' 입력한 문장중 가장 긴 문장에 맞춰 패딩 토큰을 추가

tokenizer_result = tokenizer( ['첫 번째 문장은 짧다.', '두 번째 문장은 첫 번째 문장보다 더 길다.'], padding='longest' )['input_ids']
# {'input_ids': [[0, 1656, 1141, 3135, 6265, 2073, 1599, 2062, 18, 2, 1, 1, 1, 1, 1, 1, 1], [0, 864, 1141, 3135, 6265, 2073, 1656, 1141, 3135, 6265, 2178, 2062, 831, 647, 2062, 18, 2]], 
# 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 
# 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
print(tokenizer( ['첫 번째 문장은 짧다.', '두 번째 문장은 첫 번째 문장보다 더 길다.'], padding='longest' ))

tokenizer.batch_decode(tokenizer_result)


{'input_ids': [[0, 1656, 1141, 3135, 6265, 2073, 1599, 2062, 18, 2, 1, 1, 1, 1, 1, 1, 1], [0, 864, 1141, 3135, 6265, 2073, 1656, 1141, 3135, 6265, 2178, 2062, 831, 647, 2062, 18, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


['[CLS] 첫 번째 문장은 짧다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 두 번째 문장은 첫 번째 문장보다 더 길다. [SEP]']

In [14]:
# KLUE MRC 데이터셋 다운로드 코드
from datasets import load_dataset

klue_mrc_dataset = load_dataset('klue', 'mrc') # 데이터셋 이름 klue, 서브셋 이름 mrc를 load_dataset 함수에 인자로 전달
# klue_mrc_dataset_only_train = load_dataset('klue', 'mrc', split='train') # split='train' 학습, split='validation' 검증

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
# 로컬의 데이터 활용 코드
from datasets import load_dataset

# 로컬 csv 데이터 파일을 활용
# dataset = load_dataset('csv', data_files='my_file.csv')

# 파이썬 딕셔너리 활용
from datasets import Dataset
my_dict = {'a' : [1, 2, 3]}
dataset = Dataset.from_dict(my_dict)
print(dataset)

# 판다스 데이터프레임 활용
from datasets import Dataset
import pandas as pd

df = pd.DataFrame( {'a' : [1, 2, 3]} )
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['a'],
    num_rows: 3
})
Dataset({
    features: ['a'],
    num_rows: 3
})


In [16]:
# 목표 - 한국어 기사 제목을 바탕으로 기사의 카테고리를 분류하는 텍스트 분류 모델을 학습하는 실습
# 데이터셋 준비 -> 모델, 토크나이저 로드 -> 학습(허깅페이스 트렌스포머 API) -> 모델 저장 -> 허깅페이스 허브에 업로드

In [17]:
# 데이터 준비
# 모델 학습에 사용할 연합 뉴스 데이터세 다운로드
from datasets import load_dataset

klue_tc_train = load_dataset('klue', 'ynat', split='train') # KLUE 의 YNAT 학습 및 검증 데이터셋 다운로드, train
klue_tc_eval = load_dataset('klue', 'ynat', split='validation') # validation
print(klue_tc_train)

# train 데이터는 뉴스 제목(title), 뉴스가 속한 카테고리(label) 등의 컬럼으로 이뤄진 총 45,678 개 데이터
# Dataset({
#     features: ['guid', 'title', 'label', 'url', 'date'],
#     num_rows: 45678
# })

Dataset({
    features: ['guid', 'title', 'label', 'url', 'date'],
    num_rows: 45678
})


In [18]:
# 개별 데이터 형태를 보기 위한 첫번째 데이터
klue_tc_train[0]

# 데이터의 고유 ID, {'guid': 'ynat-v1_train_00000',
# 뉴스 제목,  'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
# 속한 카테고리 ID, 'label': 3,
# 뉴스 링크, 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
# 뉴스 입력 시간, 'date': '2016.06.30. 오전 10:36'}

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [19]:
# 개별 데이터 확인, feature 속성에서 label 컬럼 데이터 확인
klue_tc_train.features['label'].names

# ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [20]:
# 데이터 전처리 - 사용하지 않는 불필요한 컬럼 제거
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])
klue_tc_train

# Dataset({
#     features: ['title', 'label'],
#     num_rows: 45678
# })

Dataset({
    features: ['title', 'label'],
    num_rows: 45678
})

In [21]:
# 데이터 전처리 - 카테고리를 문자로 표기한 label_str 컬럼 추가
klue_tc_train.features['label']
# ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

klue_tc_train.features['label'].int2str(1) # ID -> 카테고리 변환 메서드

klue_tc_label = klue_tc_train.features['label']

def make_str_label(batch):
    # print('batch : ', batch)
    batch['label_str'] = klue_tc_label.int2str(batch['label']) # ID -> 카테고리 변환 메서드
    return batch

# train map 데이터 추가
klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)

klue_tc_train[0]

{'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'label_str': '생활문화'}

In [29]:
# 데이터 전처리 - 빠른 실습 진행을 위해 학습 데이터 10,000개만 추출해 사용

# print(len(klue_tc_train), len(klue_tc_eval))

# train 10,000개 추출
train_dataset = klue_tc_train.train_test_split(test_size=200, shuffle=True, seed=42)['test']

# test 1,000개 추출
dataset = klue_tc_eval.train_test_split(test_size=50, shuffle=True, seed=42)
test_dataset = dataset['test']

# validation 1,000개 추출
valid_dataset = dataset['train'].train_test_split(test_size=50, shuffle=True, seed=42)['test']

In [30]:
# 트레이너 API를 사용해 학습 - 허깅페이스는 학습에 필요한 다양한 기능(데이터로더 준비, 로깅, 평가, 저장 등) API 제공

# Trainer를 사용한 학습 1)준비
import torch
import numpy as np
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

# title 컬럼에 토큰화 수행
def tokenize_function(examples):
    return tokenizer(examples['title'], padding='max_length', truncation=True)

# 모델 로드
model_id = 'klue/roberta-base'
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels = len(train_dataset.features['label'].names) # 분류 헤드의 분류 클래스 수를 지정
    )

# 토크나이저
tokenizer = AutoTokenizer.from_pretrained(model_id)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [31]:
# Trainer를 사용한 학습 2)학습 인자와 평가 함수 정의
# 에포크 수는 1, 배치 크기는 8, 결과는 results 폴더에 저장
# 한 에포크 학습이 끝날때마다 검증 데이터셋에 대한 평가를 수행하도록 eval_strategy = 'epoch' 설정 
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy = 'epoch',
    learning_rate=5e-5,
    push_to_hub=False
)

# 학습이 잘 이뤄지고 있는지 확인할때 사용할 평가지표를 정의
# 모델의 예측결과인 eval_pred를 입력으로 받아 예측 결과중 가장 큰값을 갖는 클래스를 np.argmax 함수로 뽑아 prediction 변수에 저장
# prediction 와 정답이 저장된 labels 가 같은 값을 갖는 결과의 비율을 정확도(accuracy)로 결과 딕셔너리에 저장 및 반환
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    prediction = np.argmax(logits, axis=-1)
    return {'accuracy' : (prediction == labels).mean()}

In [32]:
# Trainer를 사용한 학습 3)학습 진행

trainer = Trainer(
    model=model, # pre-trained model(klue/roberta-base)
    args=training_args, # 에포크 수는 1, 배치 크기는 8, 결과는 results 폴더에 저장, 한 에포크 학습이 끝날때마다 검증 데이터셋에 대한 평가를 수행하도록 eval_strategy = 'epoch' 설정
    train_dataset=train_dataset, # train dataset
    eval_dataset=valid_dataset, # validation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate(test_dataset)

/var/folders/rn/0m3hx_j94mg95hzztn5yj3940000gn/T/ipykernel_2820/608994561.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.107182,0.080000


{'eval_loss': 2.090075969696045,
 'eval_accuracy': 0.14,
 'eval_runtime': 26.864,
 'eval_samples_per_second': 1.861,
 'eval_steps_per_second': 0.261,
 'epoch': 1.0}